In [12]:
#JYSYRMSSLXBJRNGBLI

In [13]:
import tensorflow as tf
import keras
from tensorflow.keras.layers import Dense,Conv2D,Flatten,Input,Reshape
from tensorflow.keras.models import Model
from tensorflow.keras.losses import SparseCategoricalCrossentropy,MeanSquaredError,BinaryCrossentropy

In [14]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [15]:
#defining model
def model(input_shape):
    VGG = tf.keras.applications.vgg16.VGG16(include_top = False, weights = "imagenet", input_shape = input_shape)
    
    VGG.trainable = False
    
    inputs = tf.keras.Input(shape = input_shape)
    
    # Bounding_box prediction
    
    x = VGG(inputs,training = False)
    
    x1 = Conv2D(1024,(3,3), activation = "relu", padding = "same")(x)
    
    x1_a = Flatten()(x1)
    
    x1_b = Dense(512, activation = "relu")(x1_a)
    
    x2 = Conv2D(1024,(1,1), activation = "relu", padding = "same")(x1)
    
    x3 = Conv2D(256,(3,3), activation = "relu", padding = "same")(x2)
    
    x3_a = Flatten()(x3)
    
    x3_b = Dense(256, activation = "relu")(x3_a)
    
    x4 = Conv2D(128,(3,3), activation = "relu", padding = "same")(x3)
    
    x5 = Conv2D(128,(1,1), activation = "relu", padding = "same")(x4)
    
    x5_a = Flatten()(x5)
    
    x5_b = Dense(128, activation = "relu")(x5_a)
    
    
    
    #confidence
    conf_1 = Dense(1, activation = 'sigmoid', name = 'conf1')(x1_b)
    conf_2 = Dense(1, activation = 'sigmoid', name = 'conf2')(x1_b)
    conf_3 = Dense(1, activation = 'sigmoid', name = 'conf3')(x3_b)
    conf_4 = Dense(1, activation = 'sigmoid', name = 'conf4')(x3_b)
    conf_5 = Dense(1, activation = 'sigmoid', name = 'conf5')(x5_b)
    conf_6 = Dense(1, activation = 'sigmoid', name = 'conf6')(x5_b)
    
    
    #Bounding Box Prediction
    output_box1 = Dense(4, activation = 'relu', name = 'output_box1')(x1_b)
    output_box2 = Dense(4, activation = 'relu', name = 'output_box2')(x1_b)
    output_box3 = Dense(4, activation = 'relu', name = 'output_box3')(x3_b)
    output_box4 = Dense(4, activation = 'relu', name = 'output_box4')(x3_b)
    output_box5 = Dense(4, activation = 'relu', name = 'output_box5')(x5_b)
    output_box6 = Dense(4, activation = 'relu', name = 'output_box6')(x5_b)
    
    model = Model(inputs,outputs = [conf_1,conf_2,conf_3,conf_4,conf_5,conf_6,output_box1,output_box2,output_box3,output_box4,output_box5,output_box6])
    
    return model
    
    

In [ ]:
a = [1,3,5,6]

a = [10*x for x in a]
a

In [16]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pandas as pd

In [17]:
directory_im = "D:\\Train_selected_temp\\train3\\images"
directory_lb = "D:\\Train_selected_temp\\train3\\labels"

In [18]:
label_paths = []
for root, directories, files in os.walk(directory_lb):
    for filename in files:
        file_path = os.path.join(root, filename)
        label_paths.append(file_path)


In [19]:
#checking for empty CSVs
for label_path in label_paths:
    with open(label_path, 'r') as f:
            lines = f.readlines()
        # check if the file is empty
    if len(lines) == 0:
            print(f'{label_path} is empty')

In [20]:
image_paths = []
for root, directories, files in os.walk(directory_im):
    for filename in files:
        file_path = os.path.join(root, filename)
        image_paths.append(file_path)

In [21]:
image_paths

['D:\\Train_selected_temp\\train3\\images\\0000000000_jpg.rf.282296722a5e40a3b1bfea38096bf319.jpg',
 'D:\\Train_selected_temp\\train3\\images\\0000000000_jpg.rf.2cd95b07adc1aa5a82555595ee589978.jpg',
 'D:\\Train_selected_temp\\train3\\images\\0000000000_jpg.rf.546367e92931f7427eeb20ebcfffcd44.jpg',
 'D:\\Train_selected_temp\\train3\\images\\0000000000_jpg.rf.7229e60baa5c219fb17ebcc4b3095e8c.jpg',
 'D:\\Train_selected_temp\\train3\\images\\0000000000_jpg.rf.bc7c0c382b9d56553770fa0b473cf6d8.jpg',
 'D:\\Train_selected_temp\\train3\\images\\0000000001_jpg.rf.7454eca4103f6ef3abe1904ae2f57b28.jpg',
 'D:\\Train_selected_temp\\train3\\images\\0000000003_jpg.rf.1be0ea0fdc5ed0461765c55fe0a2491f.jpg',
 'D:\\Train_selected_temp\\train3\\images\\0000000003_jpg.rf.547254e9c3e3165d6310c70842100962.jpg',
 'D:\\Train_selected_temp\\train3\\images\\0000000003_jpg.rf.7a7a7d0ddef7ed8abb76b9d2aa0e6511.jpg',
 'D:\\Train_selected_temp\\train3\\images\\0000000003_jpg.rf.c385487b2ba2c90af0f5b0d81cdca9ed.jpg',


In [22]:
label_paths

['D:\\Train_selected_temp\\train3\\labels\\0000000000_jpg.rf.282296722a5e40a3b1bfea38096bf319.txt',
 'D:\\Train_selected_temp\\train3\\labels\\0000000000_jpg.rf.2cd95b07adc1aa5a82555595ee589978.txt',
 'D:\\Train_selected_temp\\train3\\labels\\0000000000_jpg.rf.546367e92931f7427eeb20ebcfffcd44.txt',
 'D:\\Train_selected_temp\\train3\\labels\\0000000000_jpg.rf.7229e60baa5c219fb17ebcc4b3095e8c.txt',
 'D:\\Train_selected_temp\\train3\\labels\\0000000000_jpg.rf.bc7c0c382b9d56553770fa0b473cf6d8.txt',
 'D:\\Train_selected_temp\\train3\\labels\\0000000001_jpg.rf.7454eca4103f6ef3abe1904ae2f57b28.txt',
 'D:\\Train_selected_temp\\train3\\labels\\0000000003_jpg.rf.1be0ea0fdc5ed0461765c55fe0a2491f.txt',
 'D:\\Train_selected_temp\\train3\\labels\\0000000003_jpg.rf.547254e9c3e3165d6310c70842100962.txt',
 'D:\\Train_selected_temp\\train3\\labels\\0000000003_jpg.rf.7a7a7d0ddef7ed8abb76b9d2aa0e6511.txt',
 'D:\\Train_selected_temp\\train3\\labels\\0000000003_jpg.rf.c385487b2ba2c90af0f5b0d81cdca9ed.txt',


In [23]:
X_train = []
for img_path in image_paths:
    image = cv2.imread(img_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    #image_resized = tf.image.resize(image, (224,224))
    X_train.append(image)
X_train = np.array(X_train, dtype = np.uint8)

In [24]:
X_train.shape

(1202, 270, 360, 3)

In [25]:
Y_train_conf = []
Y_train = [] #class
Y_train_bbox = []
for label_path in label_paths:
    conf = np.ones((6,1))   #confidence
    label = np.zeros((6,4))
    label1 = np.zeros((6,1))
    label[:,0:2] = 0.5
    label[:,2:] = 1
    lab = pd.read_csv(label_path,sep = ' ', header = None)
    rows = lab.shape[0]
    if rows>6:
        rows = 6
    if rows < 6:
        for k in range(rows,6):
            conf[k] = 0  
    label[:rows,:]= lab.iloc[:rows,-4:]
    label = [100*x for x in label]
    label1[:rows,0]= lab.iloc[:rows,0]
    Y_train.append(label1)
    Y_train_bbox.append(label)
    Y_train_conf.append(conf)
Y_train = np.array(Y_train)
Y_train_bbox = np.array(Y_train_bbox)
Y_train_conf = np.array(Y_train_conf)

In [ ]:
print(Y_train_conf)

In [ ]:
print(Y_train_bbox)


In [ ]:
print(Y_train)

In [26]:
print(Y_train.shape)
print(Y_train_bbox.shape)
print(Y_train_conf.shape)

(1202, 6, 1)
(1202, 6, 4)
(1202, 6, 1)


In [29]:
X_train_sm = X_train[:]
X_train_sm.shape

(1202, 270, 360, 3)

In [27]:
Y_train_sm = Y_train[:]
Y_train_bbox_sm = Y_train_bbox[:]
Y_train_conf_sm = Y_train_conf[:]

In [ ]:
Y_train_bbox_sm

In [28]:
print(Y_train_sm.shape)
print(Y_train_bbox_sm.shape)
print(Y_train_conf_sm.shape)

(1202, 6, 1)
(1202, 6, 4)
(1202, 6, 1)


In [ ]:
#shuffling the data
length = X_train.shape[0]
shuffle_indexes = np.random.permutation(length)
X_train_s = X_train[shuffle_indexes]
Y_train_s = Y_train[shuffle_indexes]
Y_train_bbox_s = Y_train_bbox[shuffle_indexes]
Y_train_conf_s = Y_train_conf[shuffle_indexes]

In [30]:
input_shape=(270,360,3) 
model = model(input_shape)

In [ ]:
model =keras.models.load_model('8classes.h5',compile = False)

In [31]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 270, 360, 3) 0                                            
__________________________________________________________________________________________________
vgg16 (Functional)              (None, 8, 11, 512)   14714688    input_2[0][0]                    
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 8, 11, 1024)  4719616     vgg16[0][0]                      
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 8, 11, 1024)  1049600     conv2d[0][0]                     
______________________________________________________________________________________________

In [ ]:
import tensorflow.keras.backend as K

def Error(y_true, y_pred):
    """
    Smooth L1 Loss function for bounding box regression
    """
    # Calculate absolute difference between the predicted and ground truth bounding box coordinates
    diff = K.abs(y_true - y_pred)
    
    # Calculate the Huber loss
    less_than_one = K.cast(K.less(diff, 1.0), "float32")
    loss = less_than_one * 0.5 * diff**2 + (1.0 - less_than_one) * (diff - 0.5)
    
    # Calculate the mean loss across all samples
    loss = K.mean(loss, axis=-1)
    
    return loss

In [32]:

model.compile(optimizer= tf.keras.optimizers.Adam(),
              loss={'conf1':BinaryCrossentropy(),'conf2':BinaryCrossentropy(),'conf3':BinaryCrossentropy(),'conf4':BinaryCrossentropy(),'conf5':BinaryCrossentropy(),'conf6':BinaryCrossentropy(),'output_box1': MeanSquaredError(),'output_box2': MeanSquaredError(),'output_box3': MeanSquaredError(),'output_box4':MeanSquaredError(),'output_box5':MeanSquaredError(),'output_box6':MeanSquaredError()},
              metrics={'conf1': 'accuracy','conf2': 'accuracy','conf3': 'accuracy','conf4': 'accuracy','conf5': 'accuracy','conf6': 'accuracy','output_box1': 'mse','output_box2': 'mse','output_box3': 'mse','output_box4': 'mse','output_box5': 'mse','output_box6': 'mse'})



In [33]:
model.fit(X_train_sm, [Y_train_conf_sm[:,0,:],Y_train_conf_sm[:,1,:],Y_train_conf_sm[:,2,:],Y_train_conf_sm[:,3,:],Y_train_conf_sm[:,4,:],Y_train_conf_sm[:,5,:],Y_train_bbox_sm[:,0,:],Y_train_bbox_sm[:,1,:],Y_train_bbox_sm[:,2,:],Y_train_bbox_sm[:,3,:],Y_train_bbox_sm[:,4,:],Y_train_bbox_sm[:,5,:]], batch_size=32, epochs=100)

Epoch 1/100


ResourceExhaustedError:  OOM when allocating tensor with shape[32,64,270,360] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node model/vgg16/block1_conv1/Relu (defined at C:\Users\yashl\AppData\Local\Temp\ipykernel_16172\1227237623.py:1) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_train_function_6010]

Function call stack:
train_function


SAVING THE MODEL

In [ ]:
pip install h5py pyyaml

In [ ]:
model.save('object_localization_model_n.h5')

In [ ]:
X_train.shape

In [ ]:
pred1 = model.predict(X_train[0:10])

In [ ]:
pred1

In [ ]:
img = X_train[1201]
bbox = Y_train_bbox[1201]

In [ ]:
print(img)
print(bbox[2])

In [ ]:
cut = [37.22222222, 51.11111111,  3.61111111,  7.96296296]
x1 = int(cut[0] * 3.6)
y1 = int(cut[1] * 2.7)
xw = int(cut[2] * 3.6)
yw = int(cut[3] * 2.7)
start_point = [x1 - xw, y1 - yw ]
end_point  = [x1 + xw, y1 + yw ]
for i in range(2):
    if start_point[i] <0 :
        start_point[i] = 0
    if end_point[i]<0:
        end_point[i] = 0
print(start_point)
print(end_point)

In [ ]:
image = cv2.rectangle(img, start_point, end_point,(255,0,255),2)

In [ ]:
plt.imshow(img)

In [ ]:
plt.imshow(imag)

In [ ]:
test_im = "D:\\train_big\\new_train\\2019_04_30_mlms001\\images"
test_lbl = "D:\\Train_selected\\test\\labels"

In [ ]:
image_paths_test = []
for root, directories, files in os.walk(test_im):
    for filename in files:
        file_path = os.path.join(root, filename)
        image_paths_test.append(file_path)

In [ ]:
image_paths_test

In [ ]:
X_test = []
for img_path in image_paths_test:
    image = cv2.imread(img_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_resized = tf.image.resize(image, (270,360))
    X_test.append(image_resized.numpy())
X_test = np.array(X_test, dtype = np.uint8)

In [ ]:
X_test.shape

In [ ]:
pred = model.predict(X_test[:50])

In [ ]:
pred

In [ ]:
image1 = X_test[8]

In [ ]:
plt.imshow(image1)

In [ ]:
cut = [30.365435 , 55.072426 , 26.661503 , 38.44603]
x1 = int(cut[0] * 3.6)
y1 = int(cut[1] * 2.7 )
xw = int(cut[2] * 3.6 )
yw = int(cut[3] * 2.7)
start_point = [x1 - xw, y1 - yw ]
end_point  = [x1 + xw, y1 + yw ]
for i in range(2):
    if start_point[i] <0 :
        start_point[i] = 0
    if end_point[i]<0:
        end_point[i] = 0
print(start_point)
print(end_point)

In [ ]:
image2 = cv2.rectangle(image1, start_point, end_point,(255,255,0),1)

In [ ]:
plt.imshow(image2)

In [ ]:
directory_test_im = "D:\\big_test\\Images"
directory_test_lb = "D:\\big_test\\Labels"

In [ ]:
label_paths_test = []
for root, directories, files in os.walk(directory_test_lb):
    for filename in files:
        file_path = os.path.join(root, filename)
        label_paths_test.append(file_path)
    

In [ ]:
#checking for empty CSVs
for label_path in label_paths_test:
    with open(label_path, 'r') as f:
            lines = f.readlines()
        # check if the file is empty
    if len(lines) == 0:
            print(f'{label_path} is empty')

In [ ]:
image_paths_test = []
for root, directories, files in os.walk(directory_test_im):
    for filename in files:
        file_path = os.path.join(root, filename)
        image_paths_test.append(file_path)

In [ ]:
image_paths_test

In [ ]:
label_paths_test


In [ ]:

X_test = []
for img_path in image_paths_test:
    image = cv2.imread(img_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_resized = tf.image.resize(image, (224,224))
    X_test.append(image_resized.numpy())
X_test = np.array(X_test, dtype = np.uint8)

In [ ]:
img = X_test[896]
plt.imshow(img)

In [ ]:
Y_test_pred = model.predict(X_test)

In [ ]:
Y_test_pred

In [ ]:
cut = [ 0.31026786, 0.54241071, 0.13392857, 0.24107143 ]
x1 = int(cut[0] * 224)
y1 = int(cut[1] * 224 )
xw = int(cut[2] * 224 )
yw = int(cut[3] * 224)
start_point = [x1 - xw, y1 - yw ]
end_point  = [x1 + xw, y1 + yw ]
for i in range(2):
    if start_point[i] <0 :
        start_point[i] = 0
    if end_point[i]<0:
        end_point[i] = 0
print(start_point)
print(end_point)

In [ ]:
image9 = cv2.rectangle(imageee, start_point, end_point,(255,255,0),1)

In [ ]:
plt.imshow(image9)